In [1]:
import os

import lightning as L
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint
from lightning.pytorch.tuner import Tuner
from torchvision import transforms
from torchvision.datasets import CIFAR100
from tqdm.notebook import tqdm

# Import tensorboard
%load_ext tensorboard

# Path
DATASET_PATH = os.environ.get("PATH_DATASETS", "data/")
CHECKPOINT_PATH = os.environ.get("PATH_CHECKPOINT", "saved_models/Classification/")

NUM_WORKERS = os.cpu_count()

# Setting the seed
L.seed_everything(72)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

Global seed set to 72


<Figure size 640x480 with 0 Axes>

In [2]:
max_epochs = 100
batch_size = 32
lr=1e-3
weight_decay = 1e-4
hidden_dim = 128

accelerator="gpu"
devices=[1]

## Data preparation

In [3]:
train_transforms = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(size=96, scale=(0.8, 1.0)),
        transforms.RandomGrayscale(p=0.2),
        transforms.GaussianBlur(kernel_size=9, sigma=(0.1, 0.5)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]
)

In [4]:
class CIFAR100DataModule(L.LightningDataModule):
    def __init__(self, train_transform, data_dir=DATASET_PATH, batch_size=32, num_workers=8):
        super().__init__()
        self.train_transform = train_transform
        self.val_transform = transforms.Compose(
            [
                transforms.Resize(size=96),
                transforms.ToTensor(),  
                transforms.Normalize((0.5,), (0.5,)),
            ]
        )
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
    
    def prepare_data(self):
        CIFAR100(root=self.data_dir, train=True, download=False)
        CIFAR100(root=self.data_dir, train=False, download=False)
    
    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            self.train_dset = CIFAR100(root=self.data_dir, train=True,
                                               transform=self.train_transform)
            self.val_dset = CIFAR100(root=self.data_dir, train=False,
                                             transform=self.val_transform)
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dset, batch_size=self.batch_size,
                                           num_workers=self.num_workers, pin_memory=True)


    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dset, batch_size=self.batch_size,
                                           num_workers=self.num_workers, pin_memory=True)

In [5]:
dm = CIFAR100DataModule(train_transforms)
dm.prepare_data()
dm.setup()

## Model
### Resnet pretrained with SimCLR on STL10

In [6]:
class SimCLRExtractor(nn.Module):
    def __init__(self, hidden_dim = 128) -> None:
        super().__init__()
        self.convnet = torchvision.models.resnet18(pretrained=False, num_classes=4*hidden_dim)
        self.convnet.fc = nn.Sequential(
            self.convnet.fc, 
            nn.ReLU(inplace=True),
            nn.Linear(4 * hidden_dim, hidden_dim),
        )
    
    def forward(self, x):
        return self.convnet(x)

In [7]:
class LinearClassifierProtocal(L.LightningModule):
    def __init__(self, num_classes, lr, weight_decay, weight_path, hidden_dim = 128, max_epochs=100, frozen = True) -> None:
        super().__init__()
        self.save_hyperparameters()
        self.feature_extractor = SimCLRExtractor(hidden_dim=hidden_dim)
        self.feature_extractor.load_state_dict(torch.load(weight_path)["state_dict"])
        self.feature_extractor.fc = nn.Identity()
        if frozen:
            self.feature_extractor.requires_grad_(False)
        self.classification = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, num_classes),
        )

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay)
        lr_scheduler = optim.lr_scheduler.MultiStepLR(
            optimizer, milestones=[int(self.hparams.max_epochs * 0.7), int(self.hparams.max_epochs * 0.9)], gamma=0.1
        )
        return [optimizer], [lr_scheduler]

    def _calculate_loss(self, batch, mode="train"):
        imgs, labels = batch
        features = self.feature_extractor(imgs)
        preds = self.classification(features)
        loss = F.cross_entropy(preds, labels)
        acc = (preds.argmax(dim=-1) == labels).float().mean()

        self.log(mode + "_loss", loss)
        self.log(mode + "_acc", acc)
        return loss

    def training_step(self, batch, batch_idx):
        return self._calculate_loss(batch, mode="train")

    def validation_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode="val")

### Resnet baseline

In [16]:
class ResNet(L.LightningModule):
    def __init__(self, num_classes, lr, weight_decay, hidden_dim = 128, max_epochs=100, frozen = False, pretrained=False):
        super().__init__()
        self.save_hyperparameters()
        res_num_classes = 1000 if pretrained else 4 * hidden_dim
        self.feature_extractor = torchvision.models.resnet18(pretrained=pretrained, num_classes= res_num_classes) # to match the parameter size of SimCLR
        if frozen:
            self.feature_extractor.requires_grad_(False)
        self.classification = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Linear(res_num_classes, num_classes),
        )

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay)
        lr_scheduler = optim.lr_scheduler.MultiStepLR(
            optimizer, milestones=[int(self.hparams.max_epochs * 0.7), int(self.hparams.max_epochs * 0.9)], gamma=0.1
        )
        return [optimizer], [lr_scheduler]

    def _calculate_loss(self, batch, mode="train"):
        imgs, labels = batch
        features = self.feature_extractor(imgs)
        preds = self.classification(features)
        loss = F.cross_entropy(preds, labels)
        acc = (preds.argmax(dim=-1) == labels).float().mean()

        self.log(mode + "_loss", loss)
        self.log(mode + "_acc", acc)
        return loss

    def training_step(self, batch, batch_idx):
        return self._calculate_loss(batch, mode="train")

    def validation_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode="val")

## Training and validation

In [10]:
trainer_resnet = L.Trainer(
    default_root_dir=os.path.join(CHECKPOINT_PATH, "ResNet"),
    accelerator="gpu",
    devices=[1],
    max_epochs=100,
    callbacks=[
        ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc"),
        LearningRateMonitor("epoch"),
    ],
    check_val_every_n_epoch=2,
)
trainer_resnet.logger._default_hp_metric = None 

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
trainer_pretrained = L.Trainer(
    default_root_dir=os.path.join(CHECKPOINT_PATH, "Pretrained"),
    accelerator="gpu",
    devices=[1],
    max_epochs=100,
    callbacks=[
        ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc"),
        LearningRateMonitor("epoch"),
    ],
    check_val_every_n_epoch=2,
)
trainer_pretrained.logger._default_hp_metric = None 

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
resnet_frozen = ResNet(num_classes=100, lr=lr, weight_decay=weight_decay, hidden_dim=hidden_dim, max_epochs=max_epochs, frozen = True)
trainer_resnet.fit(resnet_frozen, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name              | Type       | Params
-------------------------------------------------
0 | feature_extractor | ResNet     | 11.4 M
1 | classification    | Sequential | 51.3 K
-------------------------------------------------
51.3 K    Trainable params
11.4 M    Non-trainable params
11.5 M    Total params
45.962    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Traceback (most recent call last):
  File "/home/shengxicheng/miniconda3/envs/pl/lib/python3.8/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/home/shengxicheng/miniconda3/envs/pl/lib/python3.8/multiprocessing/util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/home/shengxicheng/miniconda3/envs/pl/lib/python3.8/multiprocessing/util.py", line 133, in _remove_temp_dir
    rmtree(tempdir)
  File "/home/shengxicheng/miniconda3/envs/pl/lib/python3.8/shutil.py", line 722, in rmtree
    onerror(os.rmdir, path, sys.exc_info())
  File "/home/shengxicheng/miniconda3/envs/pl/lib/python3.8/shutil.py", line 720, in rmtree
    os.rmdir(path)
OSError: [Errno 39] Directory not empty: '/tmp/pymp-m4h2lhs3'


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [19]:
resnet_frozen_default = ResNet(num_classes=100, lr=lr, weight_decay=weight_decay, hidden_dim=hidden_dim, max_epochs=max_epochs, frozen = True, pretrained=True)
trainer_resnet.fit(resnet_frozen_default, dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name              | Type       | Params
-------------------------------------------------
0 | feature_extractor | ResNet     | 11.7 M
1 | classification    | Sequential | 100 K 
-------------------------------------------------
100 K     Trainable params
11.7 M    Non-trainable params
11.8 M    Total params
47.158    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [8]:
resnet = ResNet(num_classes=100, lr=lr, weight_decay=weight_decay, hidden_dim=hidden_dim, max_epochs=max_epochs)
trainer_resnet.fit(resnet, dm)

/home/shengxicheng/miniconda3/envs/pl/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/shengxicheng/miniconda3/envs/pl/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.5 M
---------------------------------
11.5 M    Trainable params
0         Non-trainable params
11.5 M    Total params
45.962    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [12]:
resnet_pretrained = LinearClassifierProtocal(num_classes=100, lr=lr, weight_decay=weight_decay, 
                                             hidden_dim=hidden_dim, max_epochs=max_epochs,
                                             weight_path="saved_models/ContrastiveLearning/SimCLR/lightning_logs/version_0/checkpoints/epoch=193-step=75660.ckpt")
trainer_pretrained.fit(resnet_pretrained, dm)

/home/shengxicheng/miniconda3/envs/pl/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/shengxicheng/miniconda3/envs/pl/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name              | Type            | Params
------------------------------------------------------
0 | feature_extractor | SimCLRExtractor | 11.5 M
1 | classification    | Sequential      | 12.9 K
------------------------------------------------------
12.9 K    Trainable params
11.5 M    Non-trainable params
11.5 M    Total params
46.071    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [9]:
resnet_pretrained_supervised = LinearClassifierProtocal(num_classes=100, lr=lr, weight_decay=weight_decay, 
                                             hidden_dim=hidden_dim, max_epochs=max_epochs,
                                             weight_path="saved_models/ContrastiveLearning/SimCLR/lightning_logs/version_0/checkpoints/epoch=193-step=75660.ckpt",
                                             frozen=False)
trainer_pretrained.fit(resnet_pretrained_supervised, dm)

/home/shengxicheng/miniconda3/envs/pl/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/shengxicheng/miniconda3/envs/pl/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name              | Type            | Params
------------------------------------------------------
0 | feature_extractor | SimCLRExtractor | 11.5 M
1 | classification    | Sequential      | 12.9 K
------------------------------------------------------
11.5 M    Trainable params
0         Non-trainable params
11.5 M    Total params
46.071    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]